# 대화 상태 (Conversation State)
모델 상호작용 중 대화 상태를 관리하는 방법을 알아보세요.
OpenAI는 대화 상태를 관리하는 몇 가지 방법을 제공하며, 이는 대화에서 여러 메시지 또는 차례에 걸쳐 정보를 보존하는 데 중요합니다.

각 텍스트 생성 요청은 독립적이고 상태가 지정되지 않지만(Assistants API를 사용하는 경우 제외), 텍스트 생성 요청에 추가 메시지를 매개변수로 제공하여 여러 차례에 걸친 대화를 구현할 수 있습니다. 

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
import json

client = OpenAI()

Model = "gpt-4.1-nano"

### 대화 상태 수동 관리 - chat.completions API

In [3]:
response = client.chat.completions.create(
    model=Model,
    messages=[
        {"role": "user", "content": "똑! 똑!"},
        {"role": "assistant", "content": "누구십니까?"},
        {"role": "user", "content": "배달 왔습니다."},
    ],
)

print(response.choices[0].message.content)

네, 감사합니다! 바로 받을게요.


사용자 메시지와 어시스턴트 메시지를 번갈아 사용하면 모델에 대한 한 번의 요청으로 대화의 이전 상태를 캡처할 수 있습니다.  
생성된 응답 간에 컨텍스트를 수동으로 공유하려면 모델의 이전 응답 출력을 입력으로 포함하고 해당 입력을 다음 요청에 추가하세요.  
이렇게 이전 응답을 새 요청에 추가하면 대화가 자연스럽게 느껴지고 이전 상호작용의 컨텍스트가 유지됩니다.

In [4]:
history = [
    {
        "role": "user", "content": "재미있는 농담을 해줘."
    }
]

response = client.chat.completions.create(
    model=Model,
    messages=history,
)

print(response.choices[0].message.content)

history.append(response.choices[0].message)
history.append({ "role": "user", "content": "한가지 더 해줘." })

second_response = client.chat.completions.create(
    model=Model,
    messages=history,
)

print(second_response.choices[0].message.content)

좋아! 여기 하나 웃겨줄게:

왜 컴퓨터는 바닷가에 안 가?
  
왜냐면 바닷물에 '바이러스'가 있어서!
물론이야! 또 하나 재밌는 농담 준비했어:

왜 고양이는 컴퓨터를 싫어할까?  
  
마우스를 잡으려다가 항상 끄적거리니까!


### 대화 상태용 OpenAI API
response API를 사용하면 대화 상태를 자동으로 더 쉽게 관리할 수 있으므로 대화가 진행될 때마다 수동으로 입력을 전달할 필요가 없습니다.  
Responses API는 상태를 저장하므로 대화 전반의 컨텍스트 관리가 간단한 매개변수로 가능합니다.

In [5]:
response = client.responses.create(
    model=Model,
    input="재미있는 농담을 해줘.",
)
print(response.output_text)

second_response = client.responses.create(
    model=Model,
    previous_response_id=response.id,
    input=[{"role": "user", "content": "그게 왜 재미 있는지 설명해줘"}],
)
print(second_response.output_text)

물론이죠! 여기 하나 있습니다:

왜 바다가 웃을까요?  
왜냐하면 파도가 계속 춤추고 있거든요! 😄
물론이죠! 이 농담은 말장난과 기대치의 전환에 기반하고 있어요. "파도"라는 단어가 실제로는 바다의 물결을 가리키지만, 여기서는 "파도"가 사람처럼 춤추는 것처럼 행동한다는 의도로 농담이 만들어졌어요. 즉, "파도"가 계속 춤추고 있어 "웃을" 수 있다는 비유적 표현이죠. 이처럼 일상적인 자연 현상에 인간의 행동(춤추다, 웃다)을 비유적으로 연결해서 웃음을 유도하는 것이 이 농담의 재미입니다!


### 문서/텍스트를 기반으로 챗봇이 답변하도록 하기

문서/텍스트 내용을 API 호출시 넘겨주면 우리가 원하는 답변을 유도할 수 있습니다.

In [6]:
dataset="""Document content:
질문: 인공지능 연구소의 eRecruit 시스템은 무엇입니까? 답변: 인공지능 연구소의 eRecruit 시스템은 지원자가 하나 이상의 광고된 인공지능 연구소 채용 공고에 지원서를 제출하기 위해 정기적으로 업데이트할 수 있는 개인 프로필을 만들 수 있는 온라인 시스템입니다.
질문: 인공지능 연구소의 eRecruit 시스템에 어떻게 액세스합니까? 답변: 인공지능 연구소의 eRecruit 시스템은 다음 링크를 통해 접속할 수 있습니다: https://인공지능연구소.careers.partneragency.org/erecruit.html
질문: 인공지능 연구소의 eRecruit 시스템을 사용하여 지원하는 절차는 무엇입니까? 답변: 지원 절차 지원 절차
질문: 인공지능 연구소의 eRecruit 시스템을 사용하여 온라인으로 지원해야 합니까? 답변: 모든 지원서는 인공지능 연구소의 eRecruit 시스템을 사용하여 온라인으로 제출해야 합니다. 오프라인 서면 지원서나 이메일을 통한 지원서는 접수되지 않습니다.
질문: 인공지능 연구소의 eRecruit 시스템과 호환되는 브라우저는 무엇입니까? 답변: 인공지능 연구소의 eRecruit 시스템은 Google Chrome, Internet Explorer 6 이상에 최적화되어 있습니다. 호환성 보기 모드를 사용하려면 Internet Explorer 9을 사용해야 합니다. 신청서가 성공적으로 제출되었는지 확인하려면 다음 브라우저 중 하나를 사용하는 것이 좋습니다.
질문: 내 프로필에 로그인하는 데 문제가 있습니다. 브라우저가 응답하지 않습니다. 어떻게 해야 하나요? 답변: 구인 신청을 위해 인공지능 연구소 eRecruit 프로필에 로그인하는 데 문제가 있는 경우 이는 여러 가지 이유에서 비롯될 수 있으며 그 중 일부는 인터넷 연결과 같은 인공지능 연구소의 통제 범위를 벗어납니다. 그러나 이러한 유형의 문제를 해결하려면 다음 지침을 따르는 것이 좋습니다. • 권장 브라우저(및 버전)를 사용하고 있는지 확인하십시오. • 캐시/브라우저 기록을 지웁니다. http://www.refreshyourcache.com/en/home에서 브라우저 기록을 지우는 방법에 대한 정보를 찾을 수 있습니다. 이 작업을 수행하기 전에 브라우저에서 캐시를 지울 때의 결과를 알고 있는지 확인하십시오.
질문: 인공지능 연구소의 eRecruit 시스템을 사용하는 데 도움이 필요하면 어떻게 합니까? 답변: 인공지능 연구소의 eRecruit 시스템 사용에 대한 일반적인 질문이나 지원이 필요한 경우 헬프데스크(https://info.인공지능연구소.org/sas/erecruit/Assets/HelpDesk.aspx)에 문의하세요.
질문: 왜 등록해야 합니까? 답변: 모든 지원자는 먼저 인공지능 연구소의 eRecruit 시스템에 등록해야 합니다. 등록이 완료되면 개인 정보를 입력하고 광고된 채용 공고에 지원할 수 있도록 개인 계정이 생성됩니다.
질문: 등록할 때 사용자 이름으로 무엇을 사용해야 합니까? 답변: 인공지능 연구소 eRecruit 시스템에 등록할 때 유효한 이메일 주소를 사용자 이름으로 사용하는 것이 좋습니다.
질문: 어떤 비밀번호 형식이 허용되나요? 답변: 인공지능 연구소는 강력한 비밀번호 사용을 권장합니다. 비밀번호는 최소 8자 이상이어야 하며 문자와 숫자를 조합해야 합니다.
질문: 비밀번호를 어떻게 변경할 수 있나요? 답변: 시스템에 로그인한 후 '개인 정보' 링크에서 '비밀번호 변경' 옵션을 선택하세요. '사용자 이름 또는 비밀번호를 잊으셨나요?' 링크를 클릭하고 지침에 따라 잊어버린 비밀번호를 검색하세요.
질문: 비밀번호를 잊어버렸습니다. 어떻게 해야 하나요? 답변: 비밀번호를 잊어버린 경우, 사용자 ID 또는 비밀번호 찾기 링크를 클릭하고 두 가지 옵션 중 하나를 완료하세요.
질문: 내 데이터는 안전합니까? 답변: 인공지능 연구소의 eRecruit 데이터는 개인 보안 데이터베이스에 저장되며 인공지능 연구소는 데이터의 소유자입니다. 이 시스템은 인공지능 연구소의 엄격한 보안 요구 사항을 충족합니다.
질문: 각 섹션의 모든 정보를 작성해야 합니까? 답변: 별표(*)가 표시된 모든 항목은 필수 정보이며 각 섹션별로 작성해야 합니다.
질문: 한 세션에서 모든 정보를 완료해야 합니까? 답변: 지원 과정 중 언제든지 지원서를 저장하고 나중에 계속할 수 있습니다. 섹션을 완료할 때 정보가 손실되지 않도록 정기적으로 저장 버튼을 사용하는 것이 좋습니다.
질문: 달력 기능을 사용하여 날짜를 어떻게 선택합니까? 답변: 날짜는 두 가지 방법으로 선택할 수 있습니다. A) dd/mm/yyyy 형식을 사용하여 필드에 날짜를 직접 입력하거나 B) 달력 아이콘을 클릭하여 날짜를 선택하고 연도, 월 및 일을 선택할 수 있습니다.
질문: 관련 정보가 드롭다운 옵션에 포함되어 있지 않으면 어떻게 해야 합니까? 답변: 사용 가능한 드롭다운 옵션 중에서 선택해야 합니다. 귀하의 개인 정보, 기술 및 경험에 가장 가까운 옵션을 선택하십시오.
질문: 조회 기능을 어떻게 사용합니까? 답변: 조회 기능을 사용하려면 돋보기 아이콘을 클릭하세요. 그런 다음 해당 필드에 전체 또는 부분 값을 입력하고 조회 버튼을 클릭합니다. 마지막으로 검색 결과에서 적절한 값을 선택하세요.
질문: 맞춤법 검사 기능을 사용할 수 있나요? 답변: 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 클릭 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 사전을 클릭하세요.
"""

In [7]:
response = client.responses.create(
    model=Model,
    instructions="""
        콜센터 응답자 역할을 해주시기 바랍니다. 당신의 이름은 "콜센터 도우미"입니다. 처음 대화 시작할 때는 '안녕하세요 고객님. 무엇을 도와드릴까요?'이라고 말하세요.
        당신은 주어진 정보에 대한 답변을 나에게 제공할 것입니다. 답변이 포함되지 않은 경우 "죄송합니다. 정보가 없습니다."라고 말하세요. 그 후에는 중지하세요.
        정보에 관한 질문이 아니면 답변을 거부하세요.
        """,
    input=[{"role": "user", "content": dataset}]
)

print(response.output_text)

while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() == "quit":
        break

    response = client.responses.create(
        model=Model,
        previous_response_id=response.id,
        input=[{"role": "user", "content": user_input}],
    )
    print("assistant:", response.output_text)

안녕하세요 고객님. 무엇을 도와드릴까요?


사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?


assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 검사 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 해당 텍스트 필드에서 '사전' 혹은 '맞춤법 검사' 버튼을 클릭하시면 됩니다.


사용자 입력:  각 섹션의 모든 정보를 제공해야하나요


assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 별표(*)로 표시된 모든 항목이 필수 정보이기 때문에 각 섹션의 모든 정보를 제공해야 합니다. 이를 통해 지원서가 정상적으로 접수되고 심사 절차가 원활하게 진행될 수 있습니다.


사용자 입력:  quit


------------------
(대화 예시)
```
안녕하세요 고객님. 무엇을 도와드릴까요?
사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 검사 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 해당 필드 내에서 사전 클릭 옵션을 선택하시면 됩니다.
사용자 입력:  인공지능 연구소는 어떤 비밀번호 형식을 요구하나요 
assistant: 인공지능 연구소는 강력한 비밀번호 사용을 권장하며, 비밀번호는 최소 8자 이상이어야 하고 문자와 숫자를 조합해야 합니다.
사용자 입력:  각 섹션의 모든 정보를 제공해야하나요
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 별표(*)로 표시된 모든 항목이 필수 정보로 지정되어 있으므로, 각 섹션의 모든 필수 항목을 반드시 작성해야 합니다.
사용자 입력:  강남에서 남산에 가는 교통편을 알려주세요 
assistant: 강남에서 남산으로 가는 교통편은 여러 가지가 있습니다. 대표적인 방법은 다음과 같습니다:

1. **지하철 이용**:
   - 강남역에서 2호선을 타고 '역삼역' 또는 '선릉역'에서 3호선으로 환승 → 충무로역 또는 명동역에서 하차 → 남산 방향으로 도보 혹은 버스 이용
   - 또는 강남역에서 9호선으로 환승해 '명동역' 또는 '남산촌차고지'로 이동하는 방법도 있습니다.

2. **버스 이용**:
   - 강남 일대에서 남산 순환버스(남산순환버스 또는 시내버스)를 이용할 수 있으며, 일부 버스는 남산 근처 정류장에 정차합니다.

3. **택시 또는 차량 이용**:
   - 택시를 타거나 개인 차량으로 이동하면 약 20~30분 정도 소요됩니다. 남산공원 및 케이블카 하차장 근처에 주차하거나, 케이블카를 이용하려면 남산케이블카 승강장 부근으로 이동하시면 됩니다.

시간과 교통 상황에 따라 최적의 경로를 선택하실 수 있으니, 출발 전에 교통정보 앱이나 지도 서비스를 확인하시는 것을 추천드립니다.
사용자 입력:  quit
```

### 컨텍스트 창 관리
컨텍스트 창을 이해하면 스레드된 대화를 성공적으로 생성하고 모델 상호작용 전반의 상태를 관리하는 데 도움이 됩니다.   
컨텍스트 창은 단일 요청에서 사용할 수 있는 최대 토큰 수입니다. 이 최대 토큰 수에는 입력, 출력 및 추론 토큰이 포함됩니다. 모델의 컨텍스트 창에 대한 자세한 내용은 모델 세부 정보를 참조하세요. 

다음 토큰 수가 컨텍스트 창 총계에 적용됩니다.
- 입력 토큰( Responses APIinput 에 대한 배열 에 포함하는 입력)  
- 출력 토큰(프롬프트에 대한 응답으로 생성된 토큰)  
- 추론 토큰(모델이 응답을 계획하는 데 사용됨)
  
컨텍스트 창 제한을 초과하여 생성된 토큰은 API 응답에서 잘릴 수 있습니다.

<img src="https://i.imgur.com/WcWPlmZ.png" width=500 />

## 실습 문제: "대화 흐름과 상태 유지"

다음의 요구사항에 따라, `OpenAI Responses API`를 사용하여 사용자의 대화 요청에 대해 **대화 흐름을 유지하면서 자연스럽게 응답하는 챗봇**을 만들어 보세요.

### 실습 시나리오

당신은 `디지털 헬스 상담 챗봇`을 만들고자 합니다. 사용자가 건강과 관련된 여러 질문을 연속적으로 했을 때, **이전 응답을 기억하고 연관성 있게 대화**를 이어가야 합니다.


### 주어진 초기 설정

* 사용자 첫 질문:
  `"요즘 소화가 잘 안돼요. 어떻게 해야 할까요?"`

* 이어지는 질문:
  `"그럼 매운 음식은 피해야 하나요?"`


### 구현 조건

1. Responses API를 사용하여 첫 번째 질문에 답변을 생성하세요.
2. `previous_response_id`를 활용하여 두 번째 질문을 **동일한 컨텍스트 흐름**으로 이어서 응답하도록 하세요.
3. 두 번째 응답은 **첫 번째 대답을 기억한 상태**에서 이어지는 조언처럼 들려야 합니다.
4. 사용자의 질문 외에 추가된 답변이 없도록 하세요 (챗봇이 혼자 말하지 않기).
5. 응답의 마지막에 챗봇이 “다른 증상은 없으신가요?” 와 같이 자연스럽게 다음 질문을 유도하도록 만들어 보세요.